In [1]:
# Please do not change this cell because some hidden tests might depend on it.
import os

# Otter grader does not handle ! commands well, so we define and use our
# own function to execute shell commands.
def shell(commands, warn=True):
    """Executes the string `commands` as a sequence of shell commands.
     
       Prints the result to stdout and returns the exit status. 
       Provides a printed warning on non-zero exit status unless `warn` 
       flag is unset.
    """
    file = os.popen(commands)
    print (file.read().rstrip('\n'))
    exit_status = file.close()
    if warn and exit_status != None:
        print(f"Completed with errors. Exit status: {exit_status}\n")
    return exit_status

shell("""
ls requirements.txt >/dev/null 2>&1
if [ ! $? = 0 ]; then
 rm -rf .tmp
 git clone https://github.com/cs187-2021/lab4-3.git .tmp
 mv .tmp/tests ./
 mv .tmp/requirements.txt ./
 rm -rf .tmp
fi
pip install -q -r requirements.txt
""")

In [2]:
# Initialize Otter
import otter
grader = otter.Notebook()

# CS187
## Lab 4-3 – Semantic ambiguity and quantifier scope

When discussing syntactic representations, we encountered a central issue in natural language — ambiguity. In lab 3-3, we introduced PP attachment ambiguity to show how a single sentence ("Twain bought a book for Howells") can have multiple distinct syntactic structures, each bearing a different meaning. 

Semantic ambiguity can arise even for a *single* parse of a sentence giving rise to different meanings. In this lab, we'll introduce one example of this phenomenon, quantifier scope ambiguity. We'll take a look at a sentence that elicits this type of ambiguity and propose two possible FOL representations of the sentence, confirming that they produce different truth values in a model. Then, we will use a syntactic-semantic grammar similar to those of lab 4-1 to parse and interpret the sentence, pointing out a weakness of this method.

# Preparation {-}

In [3]:
import os
import sys
import wget

import nltk

In [4]:
# Download code for augmented grammars
remote_script_dir = "https://raw.githubusercontent.com/nlp-course/data/master/scripts/"
local_script_dir = "./scripts/"

# Create and search the local script directory
os.makedirs(local_script_dir, exist_ok=True)
sys.path.insert(1, local_script_dir)

# Download files to script directory
wget.download(remote_script_dir + "trees/transform.py", out=local_script_dir)

# Import functions for transforming augmented grammars
import transform as xform

100% [................................................................................] 8506 / 8506

# The flight world model

We'll be using the flight world as the domain for the lab as we did for lab 4-1, so we provide a simple model for it here.

In [5]:
# Constants

London = "London"
NewYork = "New York"
Paris = "Paris"
Boston = "Boston"
TelAviv = "Tel Aviv"

DL10 = "DL10"
DL11 = "DL11"
DL13 = "DL13"
LY01 = "LY01"
LY12 = "LY12"

# Properties

Object = {London, NewYork, Paris, Boston, TelAviv, DL10, DL11, DL13, LY01, LY12}
Flight = {DL10, DL11, DL13, LY01, LY12}
City = {London, Paris, NewYork, Boston, TelAviv}
Capital = {London, Paris}

# Relations

Origin = {
    (DL10, London), 
    (DL11, London), 
    (DL13, Paris), 
    (LY01, Paris), 
    (LY12, London)
}
Destination = {
    (DL10, NewYork),
    (DL11, TelAviv),
    (DL13, Boston),
    (LY01, NewYork),
    (LY12, TelAviv),
}

# Quantifier meanings

Before discussing quantifier scope ambiguity, we need to establish an appropriate notion of quantifier meanings in natural language. In FOL, there are only two quantifiers, the universal quantifier $\forall$ ("for all") and the existential quantifier $\exists$ ("there exists"). In natural language, we find many more quantifiers. A few examples:

* Universal: "every", "each"
* Existential: "some", "a(n)"
* Exact counts: "one", "two", "three", "infinitely many"
* Approximate counts: "few", "many", "most", "several"
* Count bounds: "more than three", "fewer than 100"

Natural language quantifiers can be thought of as expressing a _relationship between two properties_. 

> Recall that a _property_ can be thought of as a set of objects (those bearing the property) or a function from objects to truth values (which takes those objects that bear the property to _true_ and objects not bearing the property to _false_). In our flight world, $Flight$ is a property, as is $\lambda x. Origin(x, Boston)$ ("objects originating in Boston").

Natural language quantifiers like "every" express a relationship between two properties, the _restriction_ of the quantifier and the _scope_ of the quantifier. For instance, in the sentence "every flight leaves from Boston", the restriction property is expressed by  "flight" and the scope property is expressed by "leaves from Boston". The relationship that "every" expresses is simply that _everything bearing the first property also bears the second property_. Similarly, "some" expresses the relationship that _there is something bearing the first property that also bears the second property_. And "most" expresses that _most (at least half, say) of the objects bearing the first property bear the second property_.

This way of thinking about quantifiers as expressing relationships between two properties is referred to as [_generalized quantifiers_](https://en.wikipedia.org/wiki/Generalized_quantifier), because it generalizes the FOL notion of quantifier by adding an explicit and separate restriction.

> In FOL, quantifiers $\forall$ and $\exists$ have only a scope and not a restriction. When we write $\forall x. \ldots$, the $\ldots$ is the scope of the quantifier; we can think of this as expressing that $\forall$ holds of the single property $\lambda x. \ldots$. How then can restriction properties be expressed in FOL? By folding them into the scope.
>    * "every flight leaves from Boston": $\forall x. Flight(x) \Longrightarrow Origin(x, Boston)$
>    * "some flight leaves from Boston": $\exists x. Flight(x) \land Origin(x, Boston)$
>
> Notice though that in incorporating the restriction ($Flight(x)$) into the scope of the FOL quantifier, we use different ways of combining them depending on the quantifier. That's somewhat inelegant. More importantly, for most NL quantifiers, _there is no way of encoding the relationship between restriction and scope with just a single property_, as we happen to be able to for the universal and existential quantifiers. Hence, semanticists have moved to the generalized quantifier approach to quantifier meanings, as we do here.

To allow for meanings for the first order quantifiers, we'll define some Python functions that capture this generalized quantifier approach. Each quantifier function takes the two properties, restriction and scope, and returns a truth value.

In [6]:
def for_all(R_property, S_property):
    """Returns `true` just in case all objects bearing `R_property`
    also have `S_property`."""
    restriction_objects = [x for x in Object if R_property(x)]
    scope_objects = [x for x in Object if S_property(x)]
    return all([x in scope_objects for x in restriction_objects])

def there_exists(R_property, S_property):
    """Returns `true` just in case at least one object bearing `R_property`
    also has `S_property`."""
    restriction_objects = [x for x in Object if R_property(x)]
    scope_objects = [x for x in Object if S_property(x)]
    return any([x in scope_objects for x in restriction_objects])

We can see how these work by verifying that it's not the case that all flights originate in London, but some flight does.

In [7]:
## expressing "every flight leaves from London" -- should be false
for_all(lambda x: x in Flight, lambda y: (y, London) in Origin)

False

In [8]:
## expressing "some flight leaves from London" -- should be true
there_exists(lambda x: x in Flight, lambda y: (y, London) in Origin)

True

Now it's your turn. Implement a generalized quantifier for the English determiner "two", which holds if exactly two of the objects in the restriction are in the scope as well.

> **Hint:** we used `Object` to store all objects.

<!--
BEGIN QUESTION
name: two_quantifier
-->

In [10]:
#TODO - Implement the `two` function
def two(R_property, S_property):
    """Returns `true` just in case exactly two objects bearing `R_property`
    also have `S_property`."""
    restriction_objects = [x for x in Object if R_property(x)]
    scope_objects = [x for x in Object if S_property(x)]
    return sum([x in scope_objects for x in restriction_objects]) == 2

In [11]:
grader.check("two_quantifier")

All tests passed!

We can use this implementation of the quantifier to demonstrate that two flights have New York as destination:

In [12]:
## expressing "two flights go to New York" -- should be true
two(lambda x: x in Flight, lambda y: (y, NewYork) in Destination)

True

Express and test the proposition that two flights originating in London have a capital as destination. You'll end up using two quantifiers.
<!--
BEGIN QUESTION
name: two_london_capital
-->

In [22]:
#TODO - express "there are two flights that leave from London and go to a capital" -- should be false
two_london_capital = two(lambda x: x in Flight and (x,London) in Origin, lambda y: there_exists(lambda c: c in Capital, lambda z:(y,z) in Destination))
two_london_capital

False

In [16]:
grader.check("two_london_capital")

All tests passed!

Express and test the proposition that two flights originating in a capital have New York as a destination.
<!--
BEGIN QUESTION
name: two_capital_newyork
-->

In [23]:
#TODO - express "there are two flights that leave from a capital and go to New York"
two_capital_newyork = two(lambda x: x in Flight and there_exists(lambda c: c in Capital, lambda z: (x,z) in Origin), lambda y: (y,NewYork) in Destination)
two_capital_newyork

True

In [18]:
grader.check("two_capital_newyork")

All tests passed!

# Quantifier scope ambiguity

Consider the sentence

* every flight leaves from a capital

Under one reading of this sentence, it is true just in case each flight leaves from a (possibly different) city that is a capital. This interpretation can be represented with the following FOL formula:

$$ \forall x. (Flight(x) \implies \exists y. (Capital(y) \land Origin(x, y))) \tag{1}$$

However, there is another possible reading of this sentence. A less intuitive yet possible interpretation could be that we are asking if there is a single capital from which all flights leave. This interpretation gives rise to a different FOL representation: 

$$ \exists y. (Capital(y) \land \forall x. (Flight(x) \implies Origin(x, y))) \tag{2} $$

This type of ambiguity is referred to as _quantifier scope ambiguity_, and appears almost inevitably when a sentence has multiple quantified noun phrases. 

> Other scope-taking elements can also give rise to scope ambiguities. For instance, modals (like "may", "must", "can"), and negation ("not") also have scope and can engender ambiguities. For instance, the sentence "you may not go" arguably has two readings, paraphrasable as "you have permission to not go" (that is, you may stay) and "you do not have permission to go" (that is, you must stay). Examining these further kinds of ambiguity is beyond the scope (so to speak) of this lab.

It is easy to see that both representations have the same components, just structured differently. The different ordering possibilities of the quantifiers is what generates the ambiguity. In representation (1), $\forall x$ has the outer scope, while in representation (2), $\exists y$ has the outer scope. Reflecting this ordering, the two readings are sometimes referred to as the *AE reading* and the *EA reading*, respectively.

> As another example of the phenomenon, consider the old joke: In my town, a person is mugged every 15 minutes...and boy is he getting tired of it.

Express the two readings of the sentence 

* every flight leaves from a capital

using the Python implementation of generalized quantifiers. Start with the AE reading.
<!--
BEGIN QUESTION
name: ambiguity_AE
-->

In [26]:
AE_reading = for_all(lambda x: x in Flight, lambda y: there_exists(lambda c: c in Capital, lambda z:(y,z) in Origin))
AE_reading

True

In [ ]:
grader.check("ambiguity_AE")

Now do the same for the EA reading.
<!--
BEGIN QUESTION
name: ambiguity_EA
-->

In [29]:
EA_reading = there_exists(lambda c: c in Capital, lambda y:for_all(lambda x: x in Flight, lambda z:(z,y) in Origin))
EA_reading

False

In [30]:
grader.check("ambiguity_EA")

All tests passed!

If you've implemented the two readings correctly, you have confirmed that the two readings of the ambiguous sentence are indeed different; they generate different answers.

# A compositional semantics for quantifiers

Instead of manually constructing a meaning representation as we did, let's use a syntactic-semantic grammar to perform this interpretation process. We will use the following augmented grammar: 
<!--
BEGIN QUESTION
name: complete_grammar
-->

In [58]:
grammar_spec = """ 
  S -> NP VP                  :  lambda NP, VP: NP(VP)
  
  VP -> V NP                  :  (lambda V, NP: lambda x: NP(V(x)))

  V -> 'leaves' 'from'        :  lambda: lambda Subj: lambda Obj: (Subj, Obj) in Origin
     | 'leave' 'from'
     | 'goes' 'to'            :  lambda: lambda Subj: lambda Obj: (Subj, Obj) in Destination
     | 'go' 'to'

  S_COMP -> 'that' VP         :  lambda X: X

  NP -> DET NOM               :  lambda DET, NOM: DET(NOM)
  NOM -> NOM S_COMP           :  lambda N, S: N and S
  NOM -> N                    :  lambda X: X
  
  NP -> 'New' 'York'          :  lambda: lambda P: P(NewYork)
      | 'Paris'               :  lambda: lambda P: P(Paris)
      | 'Tel' 'Aviv'          :  lambda: lambda P: P(TelAviv)
      | 'London'              :  lambda: lambda P: P(London)
      | 'Boston'              :  lambda: lambda P: P(Boston)

  N -> 'flight' | 'flights'   :  lambda: lambda x: x in Flight
     | 'capital' | 'capitals' :  lambda: lambda x: x in Capital
     | 'city' | 'cities'      :  lambda: lambda x: x in City

  DET -> 'every'              :  lambda: lambda R: lambda S: for_all(R, S)
       | 'a'                  :  lambda: lambda R: lambda S: there_exists(R, S)
       | 'two'                :  lambda: lambda R: lambda S: two(R,S)
"""

Some things to note about the grammar:

* Some of the nonterminals may be unfamiliar. In particular, the nonterminal `S_COMP` is for complementized relative clauses, phrases like "that leaves from Boston" in the sentence "every flight _that leaves from Boston_ goes to New York".

* Some of the productions have no augmentation. You may recall that in this format, productions with no explicit augmentation just use the one for the preceding production. Thus the "goes to" and "go to" rules have the same augmentation.

* Some of the augmentations (those involving `S_COMP` and the determiner `two`) raise `NotImplementedError`. We'll have you fill those in in a bit. Hold off for now.

* As described in the introductory lecture, this grammar makes use of Richard Montague's idea that noun phrase meanings should apply to verb phrase meanings, rather than the other way around. By so doing, we allow for quantifiers within noun phrases to scope over their verb phrases. But this approach necessitates changing the types of even simple noun phrases like "Boston". Instead of denoting an object, it too must denote a function from properties to truth values, for example, $\lambda P. P(Boston)$. 

We parse the grammar specification to extract an NLTK grammar and the augmentation dictionary, and construct a parser for the grammar.

In [59]:
grammar, augmentations = xform.parse_augmented_grammar(grammar_spec, globals=globals())
parser = nltk.parse.BottomUpChartParser(grammar)

Let's create a syntactic parse tree of the sentence we've been working with: 

In [60]:
sentence = "every flight leaves from a capital".split()
parses = [p for p in parser.parse(sentence)]
for tree in parses:
    tree.pretty_print()

                   S                                
        ___________|______________                   
       |                          VP                
       |                  ________|_______           
       NP                |                NP        
   ____|____             |             ___|_____     
  |        NOM           |            |        NOM  
  |         |            |            |         |    
 DET        N            V           DET        N   
  |         |       _____|___         |         |    
every     flight leaves     from      a      capital



To carry out the semantic interpretation process, we will use the same notation as in lab 4-1, where the variable `A__some_words` will be given the meaning for the constituent with nonterminal `A` with the span "some words". We start building the semantic representation by applying the semantic composition rules from the syntactic-semantic grammar to the meanings of the subconstituents bottom up. Here are the first few steps:

In [61]:
DET__every = (lambda: lambda R: lambda S: for_all(R, S)) () # lambda R: lambda S: for_all(R, S)
N__flight = (lambda: lambda x: x in Flight) () # lambda x: x in Flight
NOM__flight = (lambda X: X) (N__flight) # lambda x: x in Flight
NP__every_flight = (lambda DET, NN: DET(NN)) (DET__every, NOM__flight) # lambda S: for_all(lambda x: x in Flight, S)

Now you complete the derivation, computing the meanings for each of the remaining constituents.
<!--
BEGIN QUESTION
name: complete_derivation
-->

In [62]:
DET__a = (lambda: lambda R: lambda S: there_exists(R, S)) ()
N__capital = (lambda: lambda x: x in Capital) ()
NOM__capital = (lambda X: X) (N__capital)
NP__a_capital = (lambda DET, NOM: DET(NOM)) (DET__a, NOM__capital)
V__leaves_from = (lambda: lambda Subj: lambda Obj: (Subj, Obj) in Origin) ()
VP__leaves_from_a_capital = (lambda V, NP: lambda x: NP(V(x))) (V__leaves_from,NP__a_capital)
S__every_flight_leaves_from_a_capital = (lambda NP, VP: NP(VP)) (NP__every_flight, VP__leaves_from_a_capital)

In [63]:
grader.check("complete_derivation")

All tests passed!

Now that you've completed the semantic derivation, let's see what truth value the model provides for this sentence.

In [64]:
S__every_flight_leaves_from_a_capital

True

<!-- BEGIN QUESTION -->

According to the result, which of the two readings for the sentence (AE or EA) did the augmented grammar produce? Can we get the other reading with the same augmented grammar? What problems do you see resulting from this behavior?

<!--
BEGIN QUESTION
name: ambiguity_open_question
manual: true
-->

The augmented grammar produced the AE reading. 

<!-- END QUESTION -->



# Extending the grammar

To make the language more interesting, we'll extend the grammar to allow for relative clauses like "that leaves from Boston" or "that goes to a capital", as well as allowing for the determiner "two". Fill in the augmentations that raise `NotImplementedError` in the grammar above to allow for these. 

In [65]:
sentence = "every flight that goes to Tel Aviv goes to a capital"
parse = list(parser.parse(sentence.split()))[0]
parse.pretty_print()

                                       S                                           
         ______________________________|_________________________                   
        NP                                                       |                 
   _____|__________                                              |                  
  |               NOM                                            VP                
  |      __________|_________                             _______|_______           
  |     |                  S_COMP                        |               NP        
  |     |      ______________|_____                      |            ___|_____     
  |    NOM    |                    VP                    |           |        NOM  
  |     |     |          __________|_______              |           |         |    
 DET    N     |         V                  NP            V          DET        N   
  |     |     |     ____|____           ___|___      ____|___        | 

To test the extended grammar, we'll make use of the `interpret` function you implemented in lab 4-2. Copy in your solution from that lab.

<!--
BEGIN QUESTION
name: grammar_extension
-->

In [66]:
def interpret(tree, augmentations):
    return augmentations[tree.productions()[0]](*[interpret(child, augmentations) for child in tree if isinstance(child, nltk.Tree)])

In [67]:
grader.check("grammar_extension")

All tests passed!

Now we can demonstrate the full range of this grammar fragment by testing out a bunch of different sentences to determine which are true and which are false.

In [68]:
def test(sentence):
  print(sentence)
  parses = parser.parse(sentence.split())
  for parse in parses:
    print(parse, "\n==>", interpret(parse, augmentations))

In [69]:
test("every flight goes to a capital")

every flight goes to a capital
(S
  (NP (DET every) (NOM (N flight)))
  (VP (V goes to) (NP (DET a) (NOM (N capital))))) 
==> False


In [70]:
test("two flights leave from Paris")

two flights leave from Paris
(S (NP (DET two) (NOM (N flights))) (VP (V leave from) (NP Paris))) 
==> True


In [71]:
test("two flights leave from London")

two flights leave from London
(S (NP (DET two) (NOM (N flights))) (VP (V leave from) (NP London))) 
==> False


In [72]:
test("every flight that goes to Tel Aviv goes to a capital")

every flight that goes to Tel Aviv goes to a capital
(S
  (NP
    (DET every)
    (NOM
      (NOM (N flight))
      (S_COMP that (VP (V goes to) (NP Tel Aviv)))))
  (VP (V goes to) (NP (DET a) (NOM (N capital))))) 
==> False


In [73]:
test("every flight that leaves from a capital goes to a capital")

every flight that leaves from a capital goes to a capital
(S
  (NP
    (DET every)
    (NOM
      (NOM (N flight))
      (S_COMP
        that
        (VP (V leaves from) (NP (DET a) (NOM (N capital)))))))
  (VP (V goes to) (NP (DET a) (NOM (N capital))))) 
==> False


In [74]:
test("every flight that goes to a capital leaves from a capital")

every flight that goes to a capital leaves from a capital
(S
  (NP
    (DET every)
    (NOM
      (NOM (N flight))
      (S_COMP that (VP (V goes to) (NP (DET a) (NOM (N capital)))))))
  (VP (V leaves from) (NP (DET a) (NOM (N capital))))) 
==> True


In [75]:
test("every flight that goes to a city leaves from a capital")

every flight that goes to a city leaves from a capital
(S
  (NP
    (DET every)
    (NOM
      (NOM (N flight))
      (S_COMP that (VP (V goes to) (NP (DET a) (NOM (N city)))))))
  (VP (V leaves from) (NP (DET a) (NOM (N capital))))) 
==> True


<!-- BEGIN QUESTION -->

# Lab debrief – for consensus submission only

**Question:** We're interested in any thoughts your group has about this lab so that we can improve this lab for later years, and to inform later labs for this year. Please list any issues that arose or comments you have to improve the lab. Useful things to comment on might include the following: 

* Was the lab too long or too short?
* Were the readings appropriate for the lab? 
* Was it clear (at least after you completed the lab) what the points of the exercises were? 
* Are there additions or changes you think would make the lab better?

but you should comment on whatever aspects you found especially positive or negative.

<!--
BEGIN QUESTION
name: open_response_debrief
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# End of Lab 4-3 {-}

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [76]:
grader.check_all()

ambiguity_AE:

    All tests passed!
    

ambiguity_EA:

    All tests passed!
    

complete_derivation:

    All tests passed!
    

grammar_extension:

    All tests passed!
    

two_capital_newyork:

    All tests passed!
    

two_london_capital:

    All tests passed!
    

two_quantifier:

    All tests passed!